In [ ]:
import numpy as np
import pandas as pd
import tushare as ts
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from scipy import stats
from datetime import datetime, timedelta
from qvix import get_qvix
import akshare as ak

In [ ]:
df2 = ak.stock_zh_index_daily_tx(symbol="sh000001")
df2.drop_duplicates(inplace=True)
df2 = df2[['amount']]

In [ ]:
qvix = get_qvix('510050')

In [ ]:
qvix.head()

In [ ]:
# 分析关键点位的技术信号与未来涨跌幅的关系
qvix = get_qvix('510050')
df = ts.get_k_data('510050', start='2005-02-23')
df['date'] = df['date'].map(lambda d: datetime.strptime(d, '%Y-%m-%d'))
df.set_index('date', inplace=True)

df['n_highest'] = df.close.rolling(60).max()
df['n_highest'] = df['n_highest'].shift(1)
df['n_lowest'] = df.close.rolling(60).min()
df['n_lowest'] = df['n_lowest'].shift(1)

df['long_signal'] = df['close'] > df['n_highest']
df['short_signal'] = df['close'] < df['n_lowest']


In [ ]:
pd.concat([df, df2], axis=1)

In [ ]:
# 分析510050跳空分布
df = ts.get_k_data('510050', start='2005-02-23')
df['date'] = df['date'].map(lambda d_str: datetime.strptime(d_str, '%Y-%m-%d'))
df['last_date'] = df['date'].shift(1)
df['last_close'] = df['close'].shift(1)
df.dropna(inplace=True)
df['gap_rate'] = (df['open'] - df['last_close']) / df['last_close']
df['interval'] = df['date'] - df['last_date']

# 筛选数据
group_labels = ['all', 'workday', 'weekend', 'holiday']
all_day = df['gap_rate']
workday = df.loc[df['interval'] == timedelta(days=1), 'gap_rate']
weekend = df.loc[df['interval'] == timedelta(days=3), 'gap_rate']
holiday = df.loc[df['interval'] > timedelta(days=3), 'gap_rate']

hist_data = [all_day, workday, weekend, holiday]
hist_data_desc = [data.describe() for data in hist_data]
desc_df = pd.concat(hist_data_desc, axis=1)
desc_df.columns = group_labels

In [ ]:
desc_df

In [ ]:
# 绘制数据分布图
fig = ff.create_distplot(hist_data, group_labels, bin_size=.001)
fig.show()

In [ ]:
# 绘制百分位
x = np.arange(101)
workday_percent = stats.scoreatpercentile(workday.values, x)
weekend_percent = stats.scoreatpercentile(weekend.values, x)
holiday_percent = stats.scoreatpercentile(holiday.values, x)

data_dict = {
    'workday': workday_percent,
    'weekend': weekend_percent,
    'holiday': holiday_percent
}

percent_df = pd.DataFrame(data_dict)
percent_df

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=x, y=workday_percent,
                         mode='lines+markers',
                         name='workday'))
fig2.add_trace(go.Scatter(x=x, y=weekend_percent,
                         mode='lines+markers',
                         name='weekend'))
fig2.add_trace(go.Scatter(x=x, y=holiday_percent,
                         mode='lines+markers',
                         name='holiday'))

fig2.show()